In [8]:
# extract wiki dump file using wikiextractor
# python -m wikiextractor.WikiExtractor <Wikipedia dump file>
# i.e. python -m wikiextractor.WikiExtractor zhwiki-20200920-pages-articles-multistream.xml.bz2

In [9]:
# clean sql data and extracted wiki data

In [10]:
import numpy as np
import pandas as pd
import random
import os
import re
from collections import Counter
import jieba
import matplotlib.pyplot as plt
from hanziconv import HanziConv

## Preprocess data

In [11]:
output_filename = "processed_data_simplified.cor"
sql_data = 'sqlResult_1558435.csv'
clean_sql_data = 'clean_sql_data.txt'

In [12]:
# clean sql data execute **once only**
# df_ = pd.read_csv("sqlResult_1558435.csv",encoding="gb18030")
# df = df_.copy()
# sql_content = df['content'].values
# with open(clean_sql_data, 'w') as f:
#     for line in sql_content:
#         f.write(str(line))

In [13]:
# special characters elimation elimination
def filter_special_characters(line):
    special_characters = re.compile("\\【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！\\\，。=？、：“”‘’￥……（）《》【】]")
    clean_line = special_characters.sub('', line)
    return clean_line

# cut words
def cut_words(line):
    return list(jieba.cut(line))

# get rid of stop words
def get_stopwords():
    stopwords = []
    with open('stop_words.txt', encoding = 'gbk') as f:
        for line in f:
            stopwords.append(line.strip())
    return stopwords

def convert2simplified(line):
    return HanziConv.toSimplified(line)

In [14]:
stopwords = get_stopwords()
def write_to_file(filename, line):
    with open(filename, 'a+') as f:
        f.write(line)

def join_cut_words(words):
    clean_words = []
    for word in words:
        if word not in stopwords:
            clean_words.append(word)
    return ' '.join(clean_words)


def process_document(filename):
    with open(filename, 'r') as f:
        for line in f:
            filtered_line = filter_special_characters(line)
            simplified_line = convert2simplified(filtered_line)
            words = cut_words(simplified_line)
            new_line = join_cut_words(words)
            write_to_file(output_filename, new_line)

In [15]:
# preprocess sql data
process_document(clean_sql_data)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/sw/3qv9ws295vsfz9kzcdgn8m4r0000gn/T/jieba.cache
Loading model cost 0.829 seconds.
Prefix dict has been built successfully.


In [16]:
# preprocess wiki data
wiki_root_dir = './text/'
all_wiki_data_dirs = os.listdir(wiki_root_dir)
for sub_dir in all_wiki_data_dirs:
    sub_dir += '/'
    files_in_sub = os.listdir(wiki_root_dir + sub_dir)
    for each_file in files_in_sub:
        process_document(wiki_root_dir + sub_dir + each_file)